
In this trading bot I recorded data for a few days worth of deviations in october 2020
Note - The dataset is not massive, but the idea is there.

Strategy: This is an future arbitrage strategy.


In [26]:
import requests
import pandas as pd
import numpy as np
import json
import csv
import itertools
import sys
import config
from config import client_id
from config import client_id2
import time

In [51]:
def queryTD(symbols_to_use, first):
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/quotes"

    #define our payload
    if(first==True):
        payload =  {'apikey':client_id2,
                'symbol':symbols_to_use}
    else:
        payload =  {'apikey':client_id2,
                'symbol':symbols_to_use}
    

    #make a request
    content = requests.get(url = endpoint, params = payload)
    return content.json()

In [54]:

def runBot(t_end):
   
    #Variable decleration
    position_open = False
    trade_dict = {
        'Symbol':[],
        'Type': [],
        'Price': [],
        'Trade_Num':[],
        'Time': []
        }

    trades_df = pd.DataFrame(trade_dict)
    mes_current_open_position = ""
    trade_num = 0
    first_id = True
    x=0
    
    
    
    print("bot is running")

    while(time.time()<t_end):  
        
        x+=1
        if(x%100 == 0):
            print("Bot is still running n={}".format(x))
        
        #get price
        try:
            res=queryTD(["/MES", "/ES"], first_id)
            mes_price=res["/MES"]["lastPriceInDouble"]
            es_price=res["/ES"]["lastPriceInDouble"]
            
            first_id = not first_id
            
#             if(abs(mes_price-es_price)>0):
#                 print("ES: {}, MES: {}, dev: {}".format(es_price, mes_price, abs(mes_price-es_price)))


            if(position_open == False):
                if(abs(mes_price-es_price)>=.50):
                    ##BUY SIGNAL
                    print("OPEN POSITION")

                    if(mes_price>es_price):
                        #to be replaced with api call
                        print("Long ES @ {}, Short MES @ {}".format(es_price,mes_price))
                        #just tracking one for efficiency
                        mes_current_open_position = "short"

                        #catalog
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/ES","Type": "BUY", "Price": es_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/MES","Type": "SELL", "Price": mes_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)
                        
#                         trades_df.loc[len(df.index)] = ["/ES", "BUY", es_price, trade_num, time.time()]
#                         trades_df.loc[len(df.index)] = ["/MES", "SELL", mes_price, trade_num, time.time()]

                    else:
                        #to be replaced with api call
                        print("Short ES @ {}, Long MES @ {}".format(es_price,mes_price))
                        #just tracking one for efficiency
                        mes_current_open_position = "long"

                        #catalog
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/ES","Type": "SELL", "Price": es_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/MES","Type": "BUY", "Price": mes_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)

                    position_open = True
            else:
                print("there is a position open")
                if(mes_price-es_price == 0):
                    print("CLOSE POSITION")
                    if(mes_current_open_position == "short"):
                        #to be replaced with api call
                        print("Close ES @ {}, Cover MES @ {}".format(es_price, mes_price))

                        #catalog
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/ES","Type": "SELL", "Price": es_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/MES","Type": "BUY", "Price": mes_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)

                    elif(mes_current_open_position == "long"):
                        #to be replaced with api call
                        print("Cover ES @ {}, Close MES @ {}".format(es_price, mes_price))

                        #catalog
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/ES","Type": "BUY", "Price": es_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)
                        trades_df = trades_df.append(pd.Series(data = {"Symbol": "/MES","Type": "SELL", "Price": mes_price, "Trade_Num": trade_num, "Time": time.time()}), ignore_index=True)


                    trade_num += 1
                    trades_df.to_csv("trades{}".format(t_end), header = True)
                    position_open = False
                    mes_current_open_position = ""
                    
        except Exception as e:
            print(e)
#             trades_df.loc[len(df.index)] = ["na", "ERROR", 0, -1, time.time()]
            time.sleep(60)
            
            
        time.sleep(1.01)
   

In [55]:
if __name__ == "__main__":
    
    runBot(1634850000)

bot is running
Bot is still running n=100
Bot is still running n=200
Bot is still running n=300
Bot is still running n=400
Bot is still running n=500
Bot is still running n=600
Bot is still running n=700
Bot is still running n=800
Bot is still running n=900
OPEN POSITION
Long ES @ 4525.0, Short MES @ 4525.5
there is a position open
CLOSE POSITION
Close ES @ 4525.0, Cover MES @ 4525.0
Bot is still running n=1000
Bot is still running n=1100
Bot is still running n=1200
OPEN POSITION
Short ES @ 4525.5, Long MES @ 4525.0
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Cover ES @ 4525.25, Close MES @ 4525.25
Bot is still running n=1300
Bot is still running n=1400
Bot is still running n=1500
Bot is still running n=1600
OPEN POSITION
Long ES @ 4526.0, Short MES @ 4526.5
there is a position open
CLOSE POSITION
Close ES @ 4526.0, Cover MES @ 4526.0
OPEN POSITION
Long ES @ 4526.25, Short MES @ 4526.75
there is a position open
CLOS

Bot is still running n=13700
OPEN POSITION
Short ES @ 4517.75, Long MES @ 4517.25
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Cover ES @ 4517.5, Close MES @ 4517.5
Bot is still running n=13800
Bot is still running n=13900
Bot is still running n=14000
Bot is still running n=14100
Bot is still running n=14200
OPEN POSITION
Short ES @ 4517.5, Long MES @ 4517.0
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Cover ES @ 4517.25, Close MES @ 4517.25
Bot is still running n=14300
OPEN POSITION
Short ES @ 4517.5, Long MES @ 4517.0
there is a position open
there is a position open
there is a position open
CLOSE POS

Bot is still running n=24900
Bot is still running n=25000
Bot is still running n=25100
Bot is still running n=25200
Bot is still running n=25300
OPEN POSITION
Long ES @ 4516.5, Short MES @ 4517.0
there is a position open
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Close ES @ 4516.75, Cover MES @ 4516.75
Bot is still running n=25400
Bot is still running n=25500
OPEN POSITION
Long ES @ 4515.75, Short MES @ 4516.25
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Close ES @ 4516.0, Cover MES @ 4516.0
Bot is still running n=25600
Bot is still running n=25700
Bot is still running n=25800
Bot is still running n=25900
Bot is still running n=26000
Bot is still running n=26100
Bot is still running n=26200
Bot is still running n=26300
Bot is still running n=26400
Bot is still running n=26500
Bot is still running n=26600
Bot is still running n=26700
Bot is still

Bot is still running n=39100
OPEN POSITION
Short ES @ 4524.25, Long MES @ 4523.75
there is a position open
CLOSE POSITION
Cover ES @ 4524.0, Close MES @ 4524.0
Bot is still running n=39200
Bot is still running n=39300
OPEN POSITION
Short ES @ 4521.5, Long MES @ 4521.0
Bot is still running n=39400
there is a position open
CLOSE POSITION
Cover ES @ 4521.0, Close MES @ 4521.0
Bot is still running n=39500
Bot is still running n=39600
Bot is still running n=39700
Bot is still running n=39800
Bot is still running n=39900
Bot is still running n=40000
Bot is still running n=40100
Bot is still running n=40200
Bot is still running n=40300
Bot is still running n=40400
Bot is still running n=40500
Bot is still running n=40600
Bot is still running n=40700
Bot is still running n=40800
OPEN POSITION
Long ES @ 4524.0, Short MES @ 4524.5
there is a position open
CLOSE POSITION
Close ES @ 4524.0, Cover MES @ 4524.0
Bot is still running n=40900
OPEN POSITION
Long ES @ 4525.25, Short MES @ 4525.75
there i

OPEN POSITION
Short ES @ 4533.0, Long MES @ 4532.5
there is a position open
CLOSE POSITION
Cover ES @ 4533.0, Close MES @ 4533.0
Bot is still running n=53200
OPEN POSITION
Short ES @ 4532.75, Long MES @ 4532.25
there is a position open
CLOSE POSITION
Cover ES @ 4532.5, Close MES @ 4532.5
OPEN POSITION
Short ES @ 4532.75, Long MES @ 4532.25
there is a position open
CLOSE POSITION
Cover ES @ 4532.75, Close MES @ 4532.75
Bot is still running n=53300
Bot is still running n=53400
Bot is still running n=53500
OPEN POSITION
Short ES @ 4531.0, Long MES @ 4530.5
there is a position open
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Cover ES @ 4530.75, Close MES @ 4530.75
OPEN POSITION
Short ES @ 4531.5, Long MES @ 4531.0
there is a position open
there is a position open
there is a position open
CLOSE POSITION
Cover ES @ 4531.5, Close MES @ 4531.5
Bot is still running n=53600
OPEN POSITION
Short ES @ 4532.5, Long MES @ 4532.0
there is a position open
t

The following are testing blocks

In [22]:
res=queryTD(["/MES", "/ES"])
mes_price=res["/MES"]
# es_price=res["/ES"]["bidPriceInDouble"]

print(mes_price)
# print("Mes Price: {}".format(mes_price) + "\t ES Price: {}".format(es_price))
# print(abs(mes_price-es_price))
# print(abs(mes_price-es_price)>.50)


{'assetType': 'FUTURE', 'assetMainType': 'FUTURE', 'symbol': '/MES', 'bidPriceInDouble': 4529.0, 'askPriceInDouble': 4529.25, 'lastPriceInDouble': 4529.0, 'bidSizeInLong': 125, 'askSizeInLong': 13, 'bidId': '?', 'askId': '?', 'totalVolume': 283508, 'lastSizeInLong': 1, 'quoteTimeInLong': 1634741603700, 'tradeTimeInLong': 1634741603572, 'highPriceInDouble': 4532.25, 'lowPriceInDouble': 4504.25, 'closePriceInDouble': 4511.25, 'exchange': 'E', 'description': 'Micro E-mini S&P 500, Dec-21', 'lastId': '?', 'openPriceInDouble': 4515.75, 'changeInDouble': 17.75, 'futurePercentChange': 0.0039, 'exchangeName': 'XCME', 'securityStatus': 'Unknown', 'openInterest': 132101, 'mark': 4529.25, 'tick': 0.25, 'tickAmount': 1.25, 'product': '/MES', 'futurePriceFormat': 'D,D', 'futureTradingHours': 'GLBX(de=1640;0=-17001600;1=r-17001600d-15551640;7=d-16401555)', 'futureIsTradable': True, 'futureMultiplier': 5.0, 'futureIsActive': True, 'futureSettlementPrice': 4511.25, 'futureActiveSymbol': '/MESZ21', 'fu

In [154]:
this_dict = {
    "Names": [],
    "Types": [],
    "Prices": []
}

this_df = pd.DataFrame(this_dict)
some_series = pd.Series(data={"Names": "Martin","Types": "Hot","Prices": 1234})
this_df = this_df.append(some_series, ignore_index=True)
some_series2 = pd.Series(data={"Names": "Alina","Types": "asdfasd","Prices": 123123123})
this_df = this_df.append(some_series2, ignore_index=True)

this_df


,Names,Types,Prices
0,Martin,Hot,1234.0
1,Alina,asdfasd,123123123.0


{'/MES': {'assetType': 'FUTURE', 'assetMainType': 'FUTURE', 'symbol': '/MES', 'bidPriceInDouble': 4520.25, 'askPriceInDouble': 4520.5, 'lastPriceInDouble': 4520.25, 'bidSizeInLong': 24, 'askSizeInLong': 36, 'bidId': '?', 'askId': '?', 'totalVolume': 12234, 'lastSizeInLong': 1, 'quoteTimeInLong': 1634775841081, 'tradeTimeInLong': 1634775837831, 'highPriceInDouble': 4527.0, 'lowPriceInDouble': 4519.0, 'closePriceInDouble': 4528.0, 'exchange': 'E', 'description': 'Micro E-mini S&P 500, Dec-21', 'lastId': '?', 'openPriceInDouble': 4524.75, 'changeInDouble': -7.75, 'futurePercentChange': -0.0017, 'exchangeName': 'XCME', 'securityStatus': 'Unknown', 'openInterest': 132101, 'mark': 4520.25, 'tick': 0.25, 'tickAmount': 1.25, 'product': '/MES', 'futurePriceFormat': 'D,D', 'futureTradingHours': 'GLBX(de=1640;0=-17001600;1=r-17001600d-15551640;7=d-16401555)', 'futureIsTradable': True, 'futureMultiplier': 5.0, 'futureIsActive': True, 'futureSettlementPrice': 4528.0, 'futureActiveSymbol': '/MESZ21'